# Investigating the cause metrics for tsdr

In [21]:
%reload_ext autoreload
%autoreload 2

In [30]:
from collections import defaultdict, OrderedDict
import pathlib

import numpy as np
import pandas as pd
import torch
import torchinfo
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

from notebooklib.save import run_tsdr, save_tsdr, load_tsdr

In [31]:
DATASET_ID = "9n6mf"
datasets_hdbscan_sbd = load_tsdr(DATASET_ID, suffix="", revert_normalized_time_series=True)

In [34]:
from eval.groundtruth import check_cause_metrics

check_results = []
dataset_by_chaos_case = {}
for record, filtered_df, anomalous_df, reduced_df in datasets_hdbscan_sbd:

    anomalous_ok, anomalous_cause_metrics = check_cause_metrics(record.pk, anomalous_df.columns.tolist(), chaos_type=record.chaos_type(), chaos_comp=record.chaos_comp(), optional_cause=True)
    reduced_ok, reduced_cause_metrics = check_cause_metrics(record.pk, reduced_df.columns.tolist(), chaos_type=record.chaos_type(), chaos_comp=record.chaos_comp(), optional_cause=True)

    check_results.append((DATASET_ID, record.target_app(), record.chaos_type(), record.chaos_comp(), record.chaos_case_num(), anomalous_ok, anomalous_cause_metrics, reduced_ok, reduced_cause_metrics))
    dataset_by_chaos_case[(record.chaos_type(), record.chaos_comp(), record.chaos_case_num())] = (record, filtered_df, anomalous_df, reduced_df)

check_results_df = pd.DataFrame(check_results, columns=["dataset_id", "target_app", "chaos_type", "chaos_comp", "chaos_case_num", "anomalous_ok", "anomalous_cause_metrics", "reduced_ok", "reduced_cause_metrics"]).set_index(["dataset_id", "target_app", "chaos_type", "chaos_comp", "chaos_case_num"]).sort_index()
with pd.option_context("display.max_rows", None):
    display(check_results_df)

anomalous_ok  \
dataset_id target_app chaos_type          chaos_comp   chaos_case_num                 
9n6mf      sock-shop  pod-cpu-hog         carts        2                       True   
                                                       4                       True   
                                          carts-db     0                       True   
                                                       1                       True   
                                                       2                       True   
                                          catalogue    0                       True   
                                                       1                       True   
                                                       4                       True   
                                          catalogue-db 1                       True   
                                                       2                       True   
                                                       3                       True   
                                                       4                       True   
                                          orders       1                       True   
                                                       2                       True   
                                          orders-db    0                       True   
                                                       1                       True   
                                                       4                       True   
                                          payment      1                       True   
                                          user         1                       True   
                                                       2                       True   
                                                       3                       True   
                                                       4                       True   
                                          user-db      0                       True   
                                                       3                       True   
                                                       4                       True   
                      pod-memory-hog      carts        0                       True   
                                                       2                       True   
                                                       3                       True   
                                                       4                       True   
                                          carts-db     0                       True   
                                                       1                       True   
                                                       2                       True   
                                                       3                       True   
                                                       4                       True   
                                          catalogue    0                       True   
                                                       1                       True   
                                                       4                       True   
                                          catalogue-db 0                       True   
                                                       1                       True   
                                                       2                       True   
                                                       3                       True   
                                                       4                       True   
                                          orders       0                       True   
                                                       1                       True   
                                                       2             

In [35]:
import math

for row in check_results_df.reset_index().itertuples():
    if row.reduced_ok:
        continue
    record, filtered_df, anomalous_df, reduced_df = dataset_by_chaos_case[
        row.chaos_type, row.chaos_comp, row.chaos_case_num,
    ]

    _target_df = anomalous_df.apply(lambda x: scipy.stats.zscore(x)).filter(regex=f"{row.chaos_comp}")
    n, ncols = 6, 3
    nrows = math.ceil((_target_df.shape[1] / n) / ncols)
    fig, axs = plt.subplots(figsize=(20, 3.0 * nrows), ncols=ncols, nrows=nrows)
    fig.suptitle(f"anomalous: {row.chaos_type}/{row.chaos_comp}/{row.chaos_case_num}")
    fig.tight_layout()
    for i, ax in zip(range(0, _target_df.shape[1], n), axs.flatten()):
        for col, ts in _target_df.iloc[:, i : i+n].items():
            ax.plot(ts, label=col)
        ax.legend(loc='upper left', fontsize=8)
    plt.show()
    plt.close(fig=fig)

    _target_reduced_df = reduced_df.apply(lambda x: scipy.stats.zscore(x)).filter(regex=f"{row.chaos_comp}")
    n, ncols = 6, 3
    nrows = math.ceil((_target_reduced_df.shape[1] / n) / ncols)
    fig, axs = plt.subplots(figsize=(20, 3.0 * nrows), ncols=ncols, nrows=nrows)
    fig.suptitle(f"reduced: {row.chaos_type}/{row.chaos_comp}/{row.chaos_case_num}")
    fig.tight_layout()
    for i, ax in zip(range(0, _target_reduced_df.shape[1], n), axs.flatten()):
        for col, ts in _target_reduced_df.iloc[:, i : i+n].items():
            ax.plot(ts, label=col)
        ax.legend(loc='upper left', fontsize=8)
    plt.show()
    plt.close(fig=fig)